In [3]:
import numpy as np
import pandas as pd
from sportsreference.ncaab.teams import Teams
from sportsreference.ncaab.teams import Team
from tqdm import tqdm
from sportsreference.ncaab.boxscore import Boxscore
import time

In [101]:
seasons = [2021]
master_array = []
for season in seasons: 
    try:
        season_team_list = Teams(str(season))
        for team in season_team_list:
            print(team, end = '\r')
            team_schedule = team.schedule
            for game in team_schedule:
                boxscore = game.boxscore_index
                master_array.append([str(season),team.name,boxscore])
    except:
        continue
            
schedule_df = pd.DataFrame(master_array, columns=['Season','TeamName','BoxscoreIndex'])
schedule_df.to_csv('2021_schedule.csv')
schedule_df

The requested page returned a valid response, but no data could be found. Has the season begun, and is the data available on www.sports-reference.com?
The requested page returned a valid response, but no data could be found. Has the season begun, and is the data available on www.sports-reference.com?
The requested page returned a valid response, but no data could be found. Has the season begun, and is the data available on www.sports-reference.com?
The requested page returned a valid response, but no data could be found. Has the season begun, and is the data available on www.sports-reference.com?
The requested page returned a valid response, but no data could be found. Has the season begun, and is the data available on www.sports-reference.com?
The requested page returned a valid response, but no data could be found. Has the season begun, and is the data available on www.sports-reference.com?
The requested page returned a valid response, but no data could be found. Has the season begun

,Season,TeamName,BoxscoreIndex
0,2021,Abilene Christian,2020-11-25-11-abilene-christian
1,2021,Abilene Christian,2020-11-27-14-austin-peay
2,2021,Abilene Christian,2020-11-28-11-nebraska-omaha
3,2021,Abilene Christian,2020-12-01-20-abilene-christian
4,2021,Abilene Christian,2020-12-05-20-abilene-christian
...,...,...,...
8226,2021,Youngstown State,2021-02-13-17-youngstown-state
8227,2021,Youngstown State,2021-02-19-17-youngstown-state
8228,2021,Youngstown State,2021-02-20-17-youngstown-state
8229,2021,Youngstown State,2021-02-25-20-youngstown-state


In [102]:
seasons = [2021]
for season in seasons: 
# will run once for each season
    full_box_df = None
    counter = 0
    season_df = schedule_df.loc[schedule_df.Season==str(season)]
    for index, row in tqdm(season_df.iterrows()):
        if counter < 500:
            box_link = row['BoxscoreIndex']
            try:
                _df = Boxscore(box_link).dataframe
                _df['season'] = season

                if full_box_df is not None:
                    box_df = pd.concat([box_df,_df],axis=0)
                    counter += 1
                else:
                    full_box_df = _df
                    counter +=1
            except:
                counter += 1
                pass
        else:
            time.sleep(np.random.randint(40,90))
            counter = 0
box_df.to_csv('2014_2015_2016_2017_2018_2019_2020_2021_games_final.csv')

8231it [48:24,  2.83it/s]


In [110]:
pd.set_option('display.max_columns', None)
box_df['losing_name'] = box_df['losing_name'].str.replace("\n",'',regex = True)
box_df['losing_name'] = box_df['losing_name'].str.replace("\t",'',regex = True)
box_df.replace('\n','',regex = True,inplace = True)
box_df.replace('\t','',regex = True,inplace = True)
box_df.replace('&amp','',regex = True,inplace = True)
box_df['home_ranking'] = box_df['home_ranking'].fillna(0)
box_df['away_ranking'] = box_df['away_ranking'].fillna(0)
len(box_df)

79985

In [111]:
box_df_ = box_df.copy()
box_df = box_df.dropna()
len(box_df)

78004

In [31]:
box_df.losing_name.unique()

array(['Abilene Christian', 'Central Arkansas', 'Houston Baptist', 'Army',
       'Air Force', 'Arkansas-Pine Bluff', 'South Dakota', 'UC-Riverside',
       'Utah State', 'UNLV', 'San Jose State', 'Wyoming', 'Boise State',
       'Coastal Carolina', 'Akron', 'Cleveland State', 'Bethune-Cookman',
       'Oral Roberts', 'Detroit', 'Coppin State', 'Marshall',
       'Ball State', 'Ohio', 'Miami (OH)', 'Eastern Michigan',
       'Central Michigan', 'Bowling Green State', 'Northern Illinois',
       'Buffalo', 'Kent State', 'Alabama A&M', 'Tennessee State',
       'Jackson State', 'Grambling', 'Mississippi Valley State',
       'Texas Southern', 'Prairie View', 'Alabama State', 'Rutgers',
       'Troy', 'Alabama-Birmingham', 'Nebraska', 'Florida A&M', 'UNC',
       'Northeastern', 'Georgia Southern', 'Chattanooga', 'UTSA', 'Tulsa',
       'Old Dominion', 'Southern Miss', 'Charlotte', 'Rice',
       'Jacksonville State', 'Chicago State', 'Western Illinois',
       'Alcorn State', 'Alabama', 

In [112]:


list_of_stats = ['team','home','opponent','date','season','pace','result','wins','win_percentage','two_point_field_goals','two_point_field_goal_percentage','two_point_field_goal_attempts','turnovers','turnover_percentage',
            'true_shooting_percentage','total_rebounds','total_rebound_percentage','three_point_field_goals','three_point_field_goal_percentage','three_point_field_goal_attempts',
            'three_point_attempt_rate','steals','steal_percentage','ranking','points','personal_fouls','offensive_rebounds','offensive_rebound_percentage','offensive_rating','minutes_played',
            'losses','free_throws','free_throw_percentage','free_throw_attempts','free_throw_attempt_rate','field_goals','field_goal_percentage','field_goal_attempts','effective_field_goal_percentage',
            'defensive_rebounds','defensive_rating','blocks','block_percentage','assists','assist_percentage']

for stat in list_of_stats[7:]:
    list_of_stats.append('opponent_'+stat)
    
stats_dict = {key:[] for key in list_of_stats}

for Team in tqdm(box_df.losing_name.unique().tolist()):

    #Home/Winner
    temp_df = box_df[(box_df.winning_name == Team) & (box_df.winner == "Home")]
    stats_dict['team'] += temp_df.winning_name.tolist()
    stats_dict['opponent']  += temp_df.losing_name.tolist()
    stats_dict['date']  += temp_df.date.tolist()
    stats_dict['pace']  += temp_df.pace.tolist()
    stats_dict['wins']  += temp_df.home_wins.tolist()
    stats_dict['win_percentage']  += temp_df.home_win_percentage.tolist()
    stats_dict['two_point_field_goals']  += temp_df.home_two_point_field_goals.tolist()
    stats_dict['two_point_field_goal_percentage']  += temp_df.home_two_point_field_goal_percentage.tolist() 
    stats_dict['two_point_field_goal_attempts']  += temp_df.home_two_point_field_goal_attempts.tolist()
    stats_dict['turnovers']  += temp_df.home_turnovers.tolist()
    stats_dict['turnover_percentage']  += temp_df.home_turnover_percentage.tolist()
    stats_dict['true_shooting_percentage']  += temp_df.home_true_shooting_percentage.tolist()
    stats_dict['total_rebounds']  += temp_df.home_total_rebounds.tolist()
    stats_dict['total_rebound_percentage']  += temp_df.home_total_rebound_percentage.tolist()
    stats_dict['three_point_field_goals']  += temp_df.home_three_point_field_goals.tolist()
    stats_dict['three_point_field_goal_percentage']  += temp_df.home_three_point_field_goal_percentage.tolist()
    stats_dict['three_point_field_goal_attempts']  += temp_df.home_three_point_field_goal_attempts.tolist()
    stats_dict['three_point_attempt_rate']  += temp_df.home_three_point_attempt_rate.tolist()
    stats_dict['steals']  += temp_df.home_steals.tolist()
    stats_dict['steal_percentage']  += temp_df.home_steal_percentage.tolist()
    stats_dict['ranking']  += temp_df.home_ranking.tolist()
    stats_dict['points']  += temp_df.home_points.tolist()
    stats_dict['personal_fouls']  += temp_df.home_personal_fouls.tolist()
    stats_dict['offensive_rebounds']  += temp_df.home_offensive_rebounds.tolist()
    stats_dict['offensive_rebound_percentage']  += temp_df.home_offensive_rebound_percentage.tolist()
    stats_dict['offensive_rating']  += temp_df.home_offensive_rating.tolist()
    stats_dict['minutes_played']  += temp_df.home_minutes_played.tolist()
    stats_dict['losses']  += temp_df.home_losses.tolist()
    stats_dict['free_throws']  += temp_df.home_free_throws.tolist()
    stats_dict['free_throw_percentage']  += temp_df.home_free_throw_percentage.tolist()
    stats_dict['free_throw_attempts']  += temp_df.home_free_throw_attempts.tolist()
    stats_dict['free_throw_attempt_rate']  += temp_df.home_free_throw_attempt_rate.tolist()
    stats_dict['field_goals']  += temp_df.home_field_goals.tolist()
    stats_dict['field_goal_percentage']  += temp_df.home_field_goal_percentage.tolist()
    stats_dict['field_goal_attempts']  += temp_df.home_field_goal_attempts.tolist()
    stats_dict['effective_field_goal_percentage']  += temp_df.home_effective_field_goal_percentage.tolist()
    stats_dict['defensive_rebounds']  += temp_df.home_defensive_rebounds.tolist()
    stats_dict['defensive_rating']  += temp_df.home_free_throws.tolist()
    stats_dict['blocks']  += temp_df.home_blocks.tolist()
    stats_dict['block_percentage']  += temp_df.home_block_percentage.tolist()
    stats_dict['assists']  += temp_df.home_assists.tolist()
    stats_dict['assist_percentage']  += temp_df.home_assist_percentage.tolist()
    stats_dict['result'] += [1] * len(temp_df)
    stats_dict['opponent_wins'] += temp_df.away_wins.tolist()
    stats_dict['opponent_win_percentage']  += temp_df.away_win_percentage.tolist()
    stats_dict['opponent_two_point_field_goals']  += temp_df.away_two_point_field_goals.tolist()
    stats_dict['opponent_two_point_field_goal_percentage']  += temp_df.away_two_point_field_goal_percentage.tolist() 
    stats_dict['opponent_two_point_field_goal_attempts']  += temp_df.away_two_point_field_goal_attempts.tolist()
    stats_dict['opponent_turnovers']  += temp_df.away_turnovers.tolist()
    stats_dict['opponent_turnover_percentage']  += temp_df.away_turnover_percentage.tolist()
    stats_dict['opponent_true_shooting_percentage']  += temp_df.away_true_shooting_percentage.tolist()
    stats_dict['opponent_total_rebounds']  += temp_df.away_total_rebounds.tolist()
    stats_dict['opponent_total_rebound_percentage']  += temp_df.away_total_rebound_percentage.tolist()
    stats_dict['opponent_three_point_field_goals']  += temp_df.away_three_point_field_goals.tolist()
    stats_dict['opponent_three_point_field_goal_percentage']  += temp_df.away_three_point_field_goal_percentage.tolist()
    stats_dict['opponent_three_point_field_goal_attempts']  += temp_df.away_three_point_field_goal_attempts.tolist()
    stats_dict['opponent_three_point_attempt_rate']  += temp_df.away_three_point_attempt_rate.tolist()
    stats_dict['opponent_steals']  += temp_df.away_steals.tolist()
    stats_dict['opponent_steal_percentage']  += temp_df.away_steal_percentage.tolist()
    stats_dict['opponent_ranking']  += temp_df.away_ranking.tolist()
    stats_dict['opponent_points']  += temp_df.away_points.tolist()
    stats_dict['opponent_personal_fouls']  += temp_df.away_personal_fouls.tolist()
    stats_dict['opponent_offensive_rebounds']  += temp_df.away_offensive_rebounds.tolist()
    stats_dict['opponent_offensive_rebound_percentage']  += temp_df.away_offensive_rebound_percentage.tolist()
    stats_dict['opponent_offensive_rating']  += temp_df.away_offensive_rating.tolist()
    stats_dict['opponent_minutes_played']  += temp_df.away_minutes_played.tolist()
    stats_dict['opponent_losses']  += temp_df.away_losses.tolist()
    stats_dict['opponent_free_throws']  += temp_df.away_free_throws.tolist()
    stats_dict['opponent_free_throw_percentage']  += temp_df.away_free_throw_percentage.tolist()
    stats_dict['opponent_free_throw_attempts']  += temp_df.away_free_throw_attempts.tolist()
    stats_dict['opponent_free_throw_attempt_rate']  += temp_df.away_free_throw_attempt_rate.tolist()
    stats_dict['opponent_field_goals']  += temp_df.away_field_goals.tolist()
    stats_dict['opponent_field_goal_percentage']  += temp_df.away_field_goal_percentage.tolist()
    stats_dict['opponent_field_goal_attempts']  += temp_df.away_field_goal_attempts.tolist()
    stats_dict['opponent_effective_field_goal_percentage']  += temp_df.away_effective_field_goal_percentage.tolist()
    stats_dict['opponent_defensive_rebounds']  += temp_df.away_defensive_rebounds.tolist()
    stats_dict['opponent_defensive_rating']  += temp_df.away_free_throws.tolist()
    stats_dict['opponent_blocks']  += temp_df.away_blocks.tolist()
    stats_dict['opponent_block_percentage']  += temp_df.away_block_percentage.tolist()
    stats_dict['opponent_assists']  += temp_df.away_assists.tolist()
    stats_dict['opponent_assist_percentage']  += temp_df.away_assist_percentage.tolist()
    stats_dict['home'] += [1] * len(temp_df)
    stats_dict['season']  += temp_df.season.tolist()

    #Away/Winner  Games won on the road
    temp_df = box_df[(box_df.winning_name == Team) & (box_df.winner == "Away")]
    stats_dict['team']  += temp_df.winning_name.tolist()
    stats_dict['opponent']  += temp_df.losing_name.tolist()
    stats_dict['date']  += temp_df.date.tolist()
    stats_dict['pace']  += temp_df.pace.tolist()
    stats_dict['wins']  += temp_df.away_wins.tolist()
    stats_dict['win_percentage']  += temp_df.away_win_percentage.tolist()
    stats_dict['two_point_field_goals']  += temp_df.away_two_point_field_goals.tolist()
    stats_dict['two_point_field_goal_percentage']  += temp_df.away_two_point_field_goal_percentage.tolist() 
    stats_dict['two_point_field_goal_attempts']  += temp_df.away_two_point_field_goal_attempts.tolist()
    stats_dict['turnovers']  += temp_df.away_turnovers.tolist()
    stats_dict['turnover_percentage']  += temp_df.away_turnover_percentage.tolist()
    stats_dict['true_shooting_percentage']  += temp_df.away_true_shooting_percentage.tolist()
    stats_dict['total_rebounds']  += temp_df.away_total_rebounds.tolist()
    stats_dict['total_rebound_percentage']  += temp_df.away_total_rebound_percentage.tolist()
    stats_dict['three_point_field_goals']  += temp_df.away_three_point_field_goals.tolist()
    stats_dict['three_point_field_goal_percentage']  += temp_df.away_three_point_field_goal_percentage.tolist()
    stats_dict['three_point_field_goal_attempts']  += temp_df.away_three_point_field_goal_attempts.tolist()
    stats_dict['three_point_attempt_rate']  += temp_df.away_three_point_attempt_rate.tolist()
    stats_dict['steals']  += temp_df.away_steals.tolist()
    stats_dict['steal_percentage']  += temp_df.away_steal_percentage.tolist()
    stats_dict['ranking']  += temp_df.away_ranking.tolist()
    stats_dict['points']  += temp_df.away_points.tolist()
    stats_dict['personal_fouls']  += temp_df.away_personal_fouls.tolist()
    stats_dict['offensive_rebounds']  += temp_df.away_offensive_rebounds.tolist()
    stats_dict['offensive_rebound_percentage']  += temp_df.away_offensive_rebound_percentage.tolist()
    stats_dict['offensive_rating']  += temp_df.away_offensive_rating.tolist()
    stats_dict['minutes_played']  += temp_df.away_minutes_played.tolist()
    stats_dict['losses']  += temp_df.away_losses.tolist()
    stats_dict['free_throws']  += temp_df.away_free_throws.tolist()
    stats_dict['free_throw_percentage']  += temp_df.away_free_throw_percentage.tolist()
    stats_dict['free_throw_attempts']  += temp_df.away_free_throw_attempts.tolist()
    stats_dict['free_throw_attempt_rate']  += temp_df.away_free_throw_attempt_rate.tolist()
    stats_dict['field_goals']  += temp_df.away_field_goals.tolist()
    stats_dict['field_goal_percentage']  += temp_df.away_field_goal_percentage.tolist()
    stats_dict['field_goal_attempts']  += temp_df.away_field_goal_attempts.tolist()
    stats_dict['effective_field_goal_percentage']  += temp_df.away_effective_field_goal_percentage.tolist()
    stats_dict['defensive_rebounds']  += temp_df.away_defensive_rebounds.tolist()
    stats_dict['defensive_rating']  += temp_df.away_free_throws.tolist()
    stats_dict['blocks']  += temp_df.away_blocks.tolist()
    stats_dict['block_percentage']  += temp_df.away_block_percentage.tolist()
    stats_dict['assists']  += temp_df.away_assists.tolist()
    stats_dict['assist_percentage']  += temp_df.away_assist_percentage.tolist()
    stats_dict['result'] += [1] * len(temp_df)
    stats_dict['opponent_wins']  += temp_df.home_wins.tolist()
    stats_dict['opponent_win_percentage']  += temp_df.home_win_percentage.tolist()
    stats_dict['opponent_two_point_field_goals']  += temp_df.home_two_point_field_goals.tolist()
    stats_dict['opponent_two_point_field_goal_percentage']  += temp_df.home_two_point_field_goal_percentage.tolist() 
    stats_dict['opponent_two_point_field_goal_attempts']  += temp_df.home_two_point_field_goal_attempts.tolist()
    stats_dict['opponent_turnovers']  += temp_df.home_turnovers.tolist()
    stats_dict['opponent_turnover_percentage']  += temp_df.home_turnover_percentage.tolist()
    stats_dict['opponent_true_shooting_percentage']  += temp_df.home_true_shooting_percentage.tolist()
    stats_dict['opponent_total_rebounds']  += temp_df.home_total_rebounds.tolist()
    stats_dict['opponent_total_rebound_percentage']  += temp_df.home_total_rebound_percentage.tolist()
    stats_dict['opponent_three_point_field_goals']  += temp_df.home_three_point_field_goals.tolist()
    stats_dict['opponent_three_point_field_goal_percentage']  += temp_df.home_three_point_field_goal_percentage.tolist()
    stats_dict['opponent_three_point_field_goal_attempts']  += temp_df.home_three_point_field_goal_attempts.tolist()
    stats_dict['opponent_three_point_attempt_rate']  += temp_df.home_three_point_attempt_rate.tolist()
    stats_dict['opponent_steals']  += temp_df.home_steals.tolist()
    stats_dict['opponent_steal_percentage']  += temp_df.home_steal_percentage.tolist()
    stats_dict['opponent_ranking']  += temp_df.home_ranking.tolist()
    stats_dict['opponent_points']  += temp_df.home_points.tolist()
    stats_dict['opponent_personal_fouls']  += temp_df.home_personal_fouls.tolist()
    stats_dict['opponent_offensive_rebounds']  += temp_df.home_offensive_rebounds.tolist()
    stats_dict['opponent_offensive_rebound_percentage']  += temp_df.home_offensive_rebound_percentage.tolist()
    stats_dict['opponent_offensive_rating']  += temp_df.home_offensive_rating.tolist()
    stats_dict['opponent_minutes_played']  += temp_df.home_minutes_played.tolist()
    stats_dict['opponent_losses']  += temp_df.home_losses.tolist()
    stats_dict['opponent_free_throws']  += temp_df.home_free_throws.tolist()
    stats_dict['opponent_free_throw_percentage']  += temp_df.home_free_throw_percentage.tolist()
    stats_dict['opponent_free_throw_attempts']  += temp_df.home_free_throw_attempts.tolist()
    stats_dict['opponent_free_throw_attempt_rate']  += temp_df.home_free_throw_attempt_rate.tolist()
    stats_dict['opponent_field_goals']  += temp_df.home_field_goals.tolist()
    stats_dict['opponent_field_goal_percentage']  += temp_df.home_field_goal_percentage.tolist()
    stats_dict['opponent_field_goal_attempts']  += temp_df.home_field_goal_attempts.tolist()
    stats_dict['opponent_effective_field_goal_percentage']  += temp_df.home_effective_field_goal_percentage.tolist()
    stats_dict['opponent_defensive_rebounds']  += temp_df.home_defensive_rebounds.tolist()
    stats_dict['opponent_defensive_rating']  += temp_df.home_free_throws.tolist()
    stats_dict['opponent_blocks']  += temp_df.home_blocks.tolist()
    stats_dict['opponent_block_percentage']  += temp_df.home_block_percentage.tolist()
    stats_dict['opponent_assists']  += temp_df.home_assists.tolist()
    stats_dict['opponent_assist_percentage']  += temp_df.home_assist_percentage.tolist()
    stats_dict['home'] += [0] * len(temp_df)
    stats_dict['season']  += temp_df.season.tolist()

    #Away/loser Games lost at home
    temp_df = box_df[(box_df.losing_name == Team) & (box_df.winner == "Away")]
    stats_dict['team']  += temp_df.losing_name.tolist()
    stats_dict['opponent']  += temp_df.winning_name.tolist()
    stats_dict['date']  += temp_df.date.tolist()
    stats_dict['pace']  += temp_df.pace.tolist()
    stats_dict['wins']  += temp_df.home_wins.tolist()
    stats_dict['win_percentage']  += temp_df.home_win_percentage.tolist()
    stats_dict['two_point_field_goals']  += temp_df.home_two_point_field_goals.tolist()
    stats_dict['two_point_field_goal_percentage']  += temp_df.home_two_point_field_goal_percentage.tolist() 
    stats_dict['two_point_field_goal_attempts']  += temp_df.home_two_point_field_goal_attempts.tolist()
    stats_dict['turnovers']  += temp_df.home_turnovers.tolist()
    stats_dict['turnover_percentage']  += temp_df.home_turnover_percentage.tolist()
    stats_dict['true_shooting_percentage']  += temp_df.home_true_shooting_percentage.tolist()
    stats_dict['total_rebounds']  += temp_df.home_total_rebounds.tolist()
    stats_dict['total_rebound_percentage']  += temp_df.home_total_rebound_percentage.tolist()
    stats_dict['three_point_field_goals']  += temp_df.home_three_point_field_goals.tolist()
    stats_dict['three_point_field_goal_percentage']  += temp_df.home_three_point_field_goal_percentage.tolist()
    stats_dict['three_point_field_goal_attempts']  += temp_df.home_three_point_field_goal_attempts.tolist()
    stats_dict['three_point_attempt_rate']  += temp_df.home_three_point_attempt_rate.tolist()
    stats_dict['steals']  += temp_df.home_steals.tolist()
    stats_dict['steal_percentage']  += temp_df.home_steal_percentage.tolist()
    stats_dict['ranking']  += temp_df.home_ranking.tolist()
    stats_dict['points']  += temp_df.home_points.tolist()
    stats_dict['personal_fouls']  += temp_df.home_personal_fouls.tolist()
    stats_dict['offensive_rebounds']  += temp_df.home_offensive_rebounds.tolist()
    stats_dict['offensive_rebound_percentage']  += temp_df.home_offensive_rebound_percentage.tolist()
    stats_dict['offensive_rating']  += temp_df.home_offensive_rating.tolist()
    stats_dict['minutes_played']  += temp_df.home_minutes_played.tolist()
    stats_dict['losses']  += temp_df.home_losses.tolist()
    stats_dict['free_throws']  += temp_df.home_free_throws.tolist()
    stats_dict['free_throw_percentage']  += temp_df.home_free_throw_percentage.tolist()
    stats_dict['free_throw_attempts']  += temp_df.home_free_throw_attempts.tolist()
    stats_dict['free_throw_attempt_rate']  += temp_df.home_free_throw_attempt_rate.tolist()
    stats_dict['field_goals']  += temp_df.home_field_goals.tolist()
    stats_dict['field_goal_percentage']  += temp_df.home_field_goal_percentage.tolist()
    stats_dict['field_goal_attempts']  += temp_df.home_field_goal_attempts.tolist()
    stats_dict['effective_field_goal_percentage']  += temp_df.home_effective_field_goal_percentage.tolist()
    stats_dict['defensive_rebounds']  += temp_df.home_defensive_rebounds.tolist()
    stats_dict['defensive_rating']  += temp_df.home_free_throws.tolist()
    stats_dict['blocks']  += temp_df.home_blocks.tolist()
    stats_dict['block_percentage']  += temp_df.home_block_percentage.tolist()
    stats_dict['assists']  += temp_df.home_assists.tolist()
    stats_dict['assist_percentage']  += temp_df.home_assist_percentage.tolist()
    stats_dict['result'] += [0] * len(temp_df)
    stats_dict['opponent_wins']  += temp_df.away_wins.tolist()
    stats_dict['opponent_win_percentage']  += temp_df.away_win_percentage.tolist()
    stats_dict['opponent_two_point_field_goals']  += temp_df.away_two_point_field_goals.tolist()
    stats_dict['opponent_two_point_field_goal_percentage']  += temp_df.away_two_point_field_goal_percentage.tolist() 
    stats_dict['opponent_two_point_field_goal_attempts']  += temp_df.away_two_point_field_goal_attempts.tolist()
    stats_dict['opponent_turnovers']  += temp_df.away_turnovers.tolist()
    stats_dict['opponent_turnover_percentage']  += temp_df.away_turnover_percentage.tolist()
    stats_dict['opponent_true_shooting_percentage']  += temp_df.away_true_shooting_percentage.tolist()
    stats_dict['opponent_total_rebounds']  += temp_df.away_total_rebounds.tolist()
    stats_dict['opponent_total_rebound_percentage']  += temp_df.away_total_rebound_percentage.tolist()
    stats_dict['opponent_three_point_field_goals']  += temp_df.away_three_point_field_goals.tolist()
    stats_dict['opponent_three_point_field_goal_percentage']  += temp_df.away_three_point_field_goal_percentage.tolist()
    stats_dict['opponent_three_point_field_goal_attempts']  += temp_df.away_three_point_field_goal_attempts.tolist()
    stats_dict['opponent_three_point_attempt_rate']  += temp_df.away_three_point_attempt_rate.tolist()
    stats_dict['opponent_steals']  += temp_df.away_steals.tolist()
    stats_dict['opponent_steal_percentage']  += temp_df.away_steal_percentage.tolist()
    stats_dict['opponent_ranking']  += temp_df.away_ranking.tolist()
    stats_dict['opponent_points']  += temp_df.away_points.tolist()
    stats_dict['opponent_personal_fouls']  += temp_df.away_personal_fouls.tolist()
    stats_dict['opponent_offensive_rebounds']  += temp_df.away_offensive_rebounds.tolist()
    stats_dict['opponent_offensive_rebound_percentage']  += temp_df.away_offensive_rebound_percentage.tolist()
    stats_dict['opponent_offensive_rating']  += temp_df.away_offensive_rating.tolist()
    stats_dict['opponent_minutes_played']  += temp_df.away_minutes_played.tolist()
    stats_dict['opponent_losses']  += temp_df.away_losses.tolist()
    stats_dict['opponent_free_throws']  += temp_df.away_free_throws.tolist()
    stats_dict['opponent_free_throw_percentage']  += temp_df.away_free_throw_percentage.tolist()
    stats_dict['opponent_free_throw_attempts']  += temp_df.away_free_throw_attempts.tolist()
    stats_dict['opponent_free_throw_attempt_rate']  += temp_df.away_free_throw_attempt_rate.tolist()
    stats_dict['opponent_field_goals']  += temp_df.away_field_goals.tolist()
    stats_dict['opponent_field_goal_percentage']  += temp_df.away_field_goal_percentage.tolist()
    stats_dict['opponent_field_goal_attempts']  += temp_df.away_field_goal_attempts.tolist()
    stats_dict['opponent_effective_field_goal_percentage']  += temp_df.away_effective_field_goal_percentage.tolist()
    stats_dict['opponent_defensive_rebounds']  += temp_df.away_defensive_rebounds.tolist()
    stats_dict['opponent_defensive_rating']  += temp_df.away_free_throws.tolist()
    stats_dict['opponent_blocks']  += temp_df.away_blocks.tolist()
    stats_dict['opponent_block_percentage']  += temp_df.away_block_percentage.tolist()
    stats_dict['opponent_assists']  += temp_df.away_assists.tolist()
    stats_dict['opponent_assist_percentage']  += temp_df.away_assist_percentage.tolist()
    stats_dict['home'] += [1] * len(temp_df)
    stats_dict['season']  += temp_df.season.tolist()

    #Home/loser Games lost on the road
    temp_df = box_df[(box_df.losing_name == Team) & (box_df.winner == "Home")]
    stats_dict['team']  += temp_df.losing_name.tolist()
    stats_dict['opponent']  += temp_df.winning_name.tolist()
    stats_dict['date']  += temp_df.date.tolist()
    stats_dict['pace']  += temp_df.pace.tolist()
    stats_dict['wins']  += temp_df.away_wins.tolist()
    stats_dict['win_percentage']  += temp_df.away_win_percentage.tolist()
    stats_dict['two_point_field_goals']  += temp_df.away_two_point_field_goals.tolist()
    stats_dict['two_point_field_goal_percentage']  += temp_df.away_two_point_field_goal_percentage.tolist() 
    stats_dict['two_point_field_goal_attempts']  += temp_df.away_two_point_field_goal_attempts.tolist()
    stats_dict['turnovers']  += temp_df.away_turnovers.tolist()
    stats_dict['turnover_percentage']  += temp_df.away_turnover_percentage.tolist()
    stats_dict['true_shooting_percentage']  += temp_df.away_true_shooting_percentage.tolist()
    stats_dict['total_rebounds']  += temp_df.away_total_rebounds.tolist()
    stats_dict['total_rebound_percentage']  += temp_df.away_total_rebound_percentage.tolist()
    stats_dict['three_point_field_goals']  += temp_df.away_three_point_field_goals.tolist()
    stats_dict['three_point_field_goal_percentage']  += temp_df.away_three_point_field_goal_percentage.tolist()
    stats_dict['three_point_field_goal_attempts']  += temp_df.away_three_point_field_goal_attempts.tolist()
    stats_dict['three_point_attempt_rate']  += temp_df.away_three_point_attempt_rate.tolist()
    stats_dict['steals']  += temp_df.away_steals.tolist()
    stats_dict['steal_percentage']  += temp_df.away_steal_percentage.tolist()
    stats_dict['ranking']  += temp_df.away_ranking.tolist()
    stats_dict['points']  += temp_df.away_points.tolist()
    stats_dict['personal_fouls']  += temp_df.away_personal_fouls.tolist()
    stats_dict['offensive_rebounds']  += temp_df.away_offensive_rebounds.tolist()
    stats_dict['offensive_rebound_percentage']  += temp_df.away_offensive_rebound_percentage.tolist()
    stats_dict['offensive_rating']  += temp_df.away_offensive_rating.tolist()
    stats_dict['minutes_played']  += temp_df.away_minutes_played.tolist()
    stats_dict['losses']  += temp_df.away_losses.tolist()
    stats_dict['free_throws']  += temp_df.away_free_throws.tolist()
    stats_dict['free_throw_percentage']  += temp_df.away_free_throw_percentage.tolist()
    stats_dict['free_throw_attempts']  += temp_df.away_free_throw_attempts.tolist()
    stats_dict['free_throw_attempt_rate']  += temp_df.away_free_throw_attempt_rate.tolist()
    stats_dict['field_goals']  += temp_df.away_field_goals.tolist()
    stats_dict['field_goal_percentage']  += temp_df.away_field_goal_percentage.tolist()
    stats_dict['field_goal_attempts']  += temp_df.away_field_goal_attempts.tolist()
    stats_dict['effective_field_goal_percentage']  += temp_df.away_effective_field_goal_percentage.tolist()
    stats_dict['defensive_rebounds']  += temp_df.away_defensive_rebounds.tolist()
    stats_dict['defensive_rating']  += temp_df.away_free_throws.tolist()
    stats_dict['blocks']  += temp_df.away_blocks.tolist()
    stats_dict['block_percentage']  += temp_df.away_block_percentage.tolist()
    stats_dict['assists']  += temp_df.away_assists.tolist()
    stats_dict['assist_percentage']  += temp_df.away_assist_percentage.tolist()
    stats_dict['result'] += [0] * len(temp_df)
    stats_dict['opponent_wins']  += temp_df.home_wins.tolist()
    stats_dict['opponent_win_percentage']  += temp_df.home_win_percentage.tolist()
    stats_dict['opponent_two_point_field_goals']  += temp_df.home_two_point_field_goals.tolist()
    stats_dict['opponent_two_point_field_goal_percentage']  += temp_df.home_two_point_field_goal_percentage.tolist() 
    stats_dict['opponent_two_point_field_goal_attempts']  += temp_df.home_two_point_field_goal_attempts.tolist()
    stats_dict['opponent_turnovers']  += temp_df.home_turnovers.tolist()
    stats_dict['opponent_turnover_percentage']  += temp_df.home_turnover_percentage.tolist()
    stats_dict['opponent_true_shooting_percentage']  += temp_df.home_true_shooting_percentage.tolist()
    stats_dict['opponent_total_rebounds']  += temp_df.home_total_rebounds.tolist()
    stats_dict['opponent_total_rebound_percentage']  += temp_df.home_total_rebound_percentage.tolist()
    stats_dict['opponent_three_point_field_goals']  += temp_df.home_three_point_field_goals.tolist()
    stats_dict['opponent_three_point_field_goal_percentage']  += temp_df.home_three_point_field_goal_percentage.tolist()
    stats_dict['opponent_three_point_field_goal_attempts']  += temp_df.home_three_point_field_goal_attempts.tolist()
    stats_dict['opponent_three_point_attempt_rate']  += temp_df.home_three_point_attempt_rate.tolist()
    stats_dict['opponent_steals']  += temp_df.home_steals.tolist()
    stats_dict['opponent_steal_percentage']  += temp_df.home_steal_percentage.tolist()
    stats_dict['opponent_ranking']  += temp_df.home_ranking.tolist()
    stats_dict['opponent_points']  += temp_df.home_points.tolist()
    stats_dict['opponent_personal_fouls']  += temp_df.home_personal_fouls.tolist()
    stats_dict['opponent_offensive_rebounds']  += temp_df.home_offensive_rebounds.tolist()
    stats_dict['opponent_offensive_rebound_percentage']  += temp_df.home_offensive_rebound_percentage.tolist()
    stats_dict['opponent_offensive_rating']  += temp_df.home_offensive_rating.tolist()
    stats_dict['opponent_minutes_played']  += temp_df.home_minutes_played.tolist()
    stats_dict['opponent_losses']  += temp_df.home_losses.tolist()
    stats_dict['opponent_free_throws']  += temp_df.home_free_throws.tolist()
    stats_dict['opponent_free_throw_percentage']  += temp_df.home_free_throw_percentage.tolist()
    stats_dict['opponent_free_throw_attempts']  += temp_df.home_free_throw_attempts.tolist()
    stats_dict['opponent_free_throw_attempt_rate']  += temp_df.home_free_throw_attempt_rate.tolist()
    stats_dict['opponent_field_goals']  += temp_df.home_field_goals.tolist()
    stats_dict['opponent_field_goal_percentage']  += temp_df.home_field_goal_percentage.tolist()
    stats_dict['opponent_field_goal_attempts']  += temp_df.home_field_goal_attempts.tolist()
    stats_dict['opponent_effective_field_goal_percentage']  += temp_df.home_effective_field_goal_percentage.tolist()
    stats_dict['opponent_defensive_rebounds']  += temp_df.home_defensive_rebounds.tolist()
    stats_dict['opponent_defensive_rating']  += temp_df.home_free_throws.tolist()
    stats_dict['opponent_blocks']  += temp_df.home_blocks.tolist()
    stats_dict['opponent_block_percentage']  += temp_df.home_block_percentage.tolist()
    stats_dict['opponent_assists']  += temp_df.home_assists.tolist()
    stats_dict['opponent_assist_percentage']  += temp_df.home_assist_percentage.tolist()
    stats_dict['home'] += [0] * len(temp_df)
    stats_dict['season']  += temp_df.season.tolist()

100%|██████████| 956/956 [00:52<00:00, 18.06it/s]


In [113]:
final_df = pd.DataFrame.from_dict(stats_dict)
final_df['date'] = pd.to_datetime(final_df['date'])
final_df.drop_duplicates(inplace = True)
final_df.sort_values('date',inplace = True)
final_df[final_df.team == 'Purdue']

,team,home,opponent,date,season,pace,result,wins,win_percentage,two_point_field_goals,two_point_field_goal_percentage,two_point_field_goal_attempts,turnovers,turnover_percentage,true_shooting_percentage,total_rebounds,total_rebound_percentage,three_point_field_goals,three_point_field_goal_percentage,three_point_field_goal_attempts,three_point_attempt_rate,steals,steal_percentage,ranking,points,personal_fouls,offensive_rebounds,offensive_rebound_percentage,offensive_rating,minutes_played,losses,free_throws,free_throw_percentage,free_throw_attempts,free_throw_attempt_rate,field_goals,field_goal_percentage,field_goal_attempts,effective_field_goal_percentage,defensive_rebounds,defensive_rating,blocks,block_percentage,assists,assist_percentage,opponent_wins,opponent_win_percentage,opponent_two_point_field_goals,opponent_two_point_field_goal_percentage,opponent_two_point_field_goal_attempts,opponent_turnovers,opponent_turnover_percentage,opponent_true_shooting_percentage,opponent_total_rebounds,opponent_total_rebound_percentage,opponent_three_point_field_goals,opponent_three_point_field_goal_percentage,opponent_three_point_field_goal_attempts,opponent_three_point_attempt_rate,opponent_steals,opponent_steal_percentage,opponent_ranking,opponent_points,opponent_personal_fouls,opponent_offensive_rebounds,opponent_offensive_rebound_percentage,opponent_offensive_rating,opponent_minutes_played,opponent_losses,opponent_free_throws,opponent_free_throw_percentage,opponent_free_throw_attempts,opponent_free_throw_attempt_rate,opponent_field_goals,opponent_field_goal_percentage,opponent_field_goal_attempts,opponent_effective_field_goal_percentage,opponent_defensive_rebounds,opponent_defensive_rating,opponent_blocks,opponent_block_percentage,opponent_assists,opponent_assist_percentage
101264,Purdue,1,Northern Kentucky,2013-11-08,2014,66.4,1,1,1.000,20.0,0.465,43.0,12,15.2,0.570,32,50.8,7.0,0.467,15.0,0.259,6,9.1,0.0,77,15,15,46.9,116.7,200.0,0,16,0.800,20,0.345,27,0.466,58,0.526,17,16,4,10.8,12,44.4,0,0.000,14.0,0.378,37.0,14,17.1,0.557,31,49.2,13.0,0.500,26.0,0.413,7,10.6,0.0,76,17,14,45.2,115.2,200.0,1,9,0.818,11,0.175,27,0.429,63,0.532,17,9,5,11.6,18,66.7
101257,Purdue,1,Central Connecticut,2013-11-13,2014,83.2,1,2,1.000,34.0,0.618,55.0,9,9.1,0.603,48,57.8,9.0,0.375,24.0,0.304,6,7.2,0.0,109,18,19,47.5,131.3,200.0,0,14,0.583,24,0.304,43,0.544,79,0.601,29,14,13,22.8,22,51.2,0,0.000,23.0,0.404,57.0,11,11.1,0.410,35,42.2,4.0,0.190,21.0,0.269,7,8.4,0.0,73,19,14,32.6,88.0,200.0,2,15,0.652,23,0.295,27,0.346,78,0.372,21,15,3,5.5,15,55.6
101268,Purdue,1,Rider,2013-11-17,2014,72.2,1,3,1.000,28.0,0.609,46.0,9,10.8,0.535,30,49.2,3.0,0.200,15.0,0.246,9,12.5,0.0,81,27,13,36.1,112.5,200.0,0,16,0.516,31,0.508,31,0.508,61,0.533,17,16,5,13.5,15,48.4,0,0.000,16.0,0.432,37.0,13,16.3,0.568,31,50.8,7.0,0.438,16.0,0.302,2,2.8,0.0,77,27,8,32.0,106.9,200.0,2,24,0.774,31,0.585,23,0.434,53,0.500,23,24,6,13.0,11,47.8
101258,Purdue,1,Eastern Illinois,2013-11-20,2014,77.3,1,4,1.000,24.0,0.585,41.0,10,11.6,0.538,40,54.1,4.0,0.200,20.0,0.328,12,15.6,0.0,83,20,11,31.4,107.8,200.0,0,23,0.676,34,0.557,28,0.459,61,0.492,29,23,7,17.1,16,57.1,2,0.500,16.0,0.390,41.0,19,21.7,0.396,34,45.9,3.0,0.176,17.0,0.293,5,6.5,0.0,55,22,10,25.6,71.4,200.0,2,14,0.583,24,0.414,19,0.328,58,0.353,24,14,8,19.5,11,57.9
101270,Purdue,1,Siena,2013-11-24,2014,83.2,1,5,1.000,22.0,0.468,47.0,15,15.9,0.503,42,51.2,5.0,0.313,16.0,0.254,6,7.2,0.0,81,19,13,32.5,97.6,200.0,0,22,0.595,37,0.587,27,0.429,63,0.468,29,22,12,19.7,11,40.7,2,0.333,27.0,0.443,61.0,14,14.6,0.440,40,48.8,1.0,0.100,10.0,0.141,10,12.0,0.0,73,25,13,31.0,88.0,200.0,4,16,0.640,25,0.352,28,0.394,71,0.401,27,16,7,14.9,15,53.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [114]:
final_df.isna().sum()

team                          0
home                          0
opponent                      0
date                          0
season                        0
                             ..
opponent_defensive_rating     0
opponent_blocks               0
opponent_block_percentage     0
opponent_assists              0
opponent_assist_percentage    0
Length: 83, dtype: int64

In [115]:
final_df.to_csv('mldata.csv')

In [116]:
agg_cols = ['abbreviation', 'assist_percentage', 'assists', 'away_losses',
       'away_wins', 'block_percentage', 'blocks', 'conference',
       'conference_losses', 'conference_wins', 'defensive_rebounds',
       'effective_field_goal_percentage', 'field_goal_attempts',
       'field_goal_percentage', 'field_goals', 'free_throw_attempt_rate',
       'free_throw_attempts', 'free_throw_percentage', 'free_throws',
       'free_throws_per_field_goal_attempt', 'games_played', 'home_losses',
       'home_wins', 'losses', 'minutes_played', 'name', 'net_rating',
       'offensive_rating', 'offensive_rebound_percentage',
       'offensive_rebounds', 'opp_assist_percentage', 'opp_assists',
       'opp_block_percentage', 'opp_blocks', 'opp_defensive_rebounds',
       'opp_effective_field_goal_percentage', 'opp_field_goal_attempts',
       'opp_field_goal_percentage', 'opp_field_goals',
       'opp_free_throw_attempt_rate', 'opp_free_throw_attempts',
       'opp_free_throw_percentage', 'opp_free_throws',
       'opp_free_throws_per_field_goal_attempt', 'opp_offensive_rating',
       'opp_offensive_rebound_percentage', 'opp_offensive_rebounds',
       'opp_personal_fouls', 'opp_points', 'opp_steal_percentage',
       'opp_steals', 'opp_three_point_attempt_rate',
       'opp_three_point_field_goal_attempts',
       'opp_three_point_field_goal_percentage', 'opp_three_point_field_goals',
       'opp_two_point_field_goal_attempts',
       'opp_two_point_field_goal_percentage', 'opp_two_point_field_goals',
       'opp_total_rebound_percentage', 'opp_total_rebounds',
       'opp_true_shooting_percentage', 'opp_turnover_percentage',
       'opp_turnovers', 'pace', 'personal_fouls', 'points',
       'simple_rating_system', 'steal_percentage', 'steals',
       'strength_of_schedule', 'three_point_attempt_rate',
       'three_point_field_goal_attempts', 'three_point_field_goal_percentage',
       'three_point_field_goals', 'two_point_field_goal_attempts',
       'two_point_field_goal_percentage', 'two_point_field_goals',
       'total_rebound_percentage', 'total_rebounds',
       'true_shooting_percentage', 'turnover_percentage', 'turnovers',
       'win_percentage', 'wins']

In [117]:
df_agg = pd.DataFrame(columns=agg_cols)
seasons = [2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021]
for season in seasons:
    print("Getting Season Data...")
    for t in tqdm(Teams(season)):
        try:
            temp_df = t.dataframe
            temp_df['season'] = season
            df_agg = pd.concat([df_agg,temp_df],axis=0)
        except:
            pass
        
df_agg

Getting Season Data...


100%|██████████| 345/345 [00:06<00:00, 56.33it/s]


Getting Season Data...


100%|██████████| 344/344 [00:08<00:00, 41.34it/s]


Getting Season Data...


...

Getting Season Data...


100%|██████████| 353/353 [00:09<00:00, 36.44it/s]


Getting Season Data...


100%|██████████| 356/356 [00:09<00:00, 37.10it/s]


,abbreviation,assist_percentage,assists,away_losses,away_wins,block_percentage,blocks,conference,conference_losses,conference_wins,defensive_rebounds,effective_field_goal_percentage,field_goal_attempts,field_goal_percentage,field_goals,free_throw_attempt_rate,free_throw_attempts,free_throw_percentage,free_throws,free_throws_per_field_goal_attempt,games_played,home_losses,home_wins,losses,minutes_played,name,net_rating,offensive_rating,offensive_rebound_percentage,offensive_rebounds,opp_assist_percentage,opp_assists,opp_block_percentage,opp_blocks,opp_defensive_rebounds,opp_effective_field_goal_percentage,opp_field_goal_attempts,opp_field_goal_percentage,opp_field_goals,opp_free_throw_attempt_rate,opp_free_throw_attempts,opp_free_throw_percentage,opp_free_throws,opp_free_throws_per_field_goal_attempt,opp_offensive_rating,opp_offensive_rebound_percentage,opp_offensive_rebounds,opp_personal_fouls,opp_points,opp_steal_percentage,opp_steals,opp_three_point_attempt_rate,opp_three_point_field_goal_attempts,opp_three_point_field_goal_percentage,opp_three_point_field_goals,opp_two_point_field_goal_attempts,opp_two_point_field_goal_percentage,opp_two_point_field_goals,opp_total_rebound_percentage,opp_total_rebounds,opp_true_shooting_percentage,opp_turnover_percentage,opp_turnovers,pace,personal_fouls,points,simple_rating_system,steal_percentage,steals,strength_of_schedule,three_point_attempt_rate,three_point_field_goal_attempts,three_point_field_goal_percentage,three_point_field_goals,two_point_field_goal_attempts,two_point_field_goal_percentage,two_point_field_goals,total_rebound_percentage,total_rebounds,true_shooting_percentage,turnover_percentage,turnovers,win_percentage,wins,season
AIR-FORCE,AIR-FORCE,68.4,494,11,3,8.7,83,mwc,10,6,731,0.540,1532,0.471,722,0.387,593,0.705,418,0.273,32,5,12,16,1295,Air Force,None,102.5,20.4,187,63.7,449,10.3,100,729,0.482,1737,0.406,705,0.326,567,0.716,406,0.234,None,34.3,381,587,2082,8.6,174,0.454,788,0.338,266,949,0.463,439,54.7,1110,0.519,16.9,407,62.5,563,2074,2.71,9.9,200,3.31,0.367,563,0.377,212,969,0.526,510,45.3,918,0.572,17.5,386,0.500,16,2011.0
AKRON,AKRON,58.3,520,9,5,9.2,129,mac,7,9,875,0.499,2073,0.430,892,0.313,649,0.704,457,0.220,36,2,14,13,1465,Akron,None,102.9,28.3,363,51.1,425,6.6,85,920,0.464,2004,0.415,832,0.383,768,0.660,507,0.253,None,32.7,425,655,2365,8.8,216,0.302,606,0.320,194,1398,0.456,638,52.1,1345,0.499,17.8,514,67.0,692,2524,2.15,9.8,240,-1.02,0.379,785,0.361,283,1288,0.473,609,47.9,1238,0.530,15.4,432,0.639,23,2011.0
ALABAMA-AM,ALABAMA-AM,53.6,339,10,4,13.1,151,swac,8,10,711,0.435,1598,0.396,633,0.418,668,0.624,417,0.261,28,4,9,15,1125,Alabama A&M,None,89.8,31.7,347,55.2,363,10.1,117,749,0.457,1594,0.413,658,0.415,662,0.624,413,0.259,None,33.6,360,565,1870,11.0,221,0.277,441,0.320,141,1153,0.448,517,51.2,1109,0.490,19.6,465,71.6,562,1808,-15.19,11.6,233,-11.75,0.272,434,0.288,125,1164,0.436,508,48.8,1058,0.472,18.8,444,0.464,13,2011.0
ALABAMA-BIRMINGHAM,ALABAMA-BIRMINGHAM,55.6,420,5,8,8.2,97,cusa,4,12,757,0.508,1725,0.438,756,0.297,513,0.717,368,0.213,31,2,13,9,1270,Alabama-Birmingham,None,105.7,33.4,352,48.5,339,9.9,102,702,0.459,1692,0.413,699,0.324,549,0.703,386,0.228,None,30.6,333,539,1940,9.3,186,0.297,503,0.310,156,1189,0.457,543,48.3,1035,0.497,16.5,385,63.1,514,2119,8.55,8.2,164,2.78,0.405,698,0.342,239,1027,0.503,517,51.7,1109,0.538,16.4,387,0.710,22,2011.0
ALABAMA-STATE,ALABAMA-STATE,55.5,404,14,4,13.1,163,swac,7,11,811,0.440,1841,0.395,728,0.473,870,0.603,525,0.285,35,3,10,18,1410,Alabama State,None,91.5,35.7,460,55.9,400,6.8,86,829,0.454,1773,0.403,715,0.515,913,0.683,624,0.352,None,33.4,406,722,2234,11.2,262,0.296,525,0.343,180,1248,0.429,535,49.3,1235,0.506,19.8,545,66.5,777,2146,-13.37,10.9,255,-10.31,0.317,583,0.283,165,1258,0.448,563,50.7,1271,0.476,19.6,551,0.486,17,2011.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [119]:
df_agg.to_csv('aggdata.csv')

In [118]:
df_agg.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3844 entries, AIR-FORCE to YOUNGSTOWN-STATE
Data columns (total 85 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   abbreviation                            3844 non-null   object 
 1   assist_percentage                       3844 non-null   float64
 2   assists                                 3844 non-null   object 
 3   away_losses                             3844 non-null   object 
 4   away_wins                               3844 non-null   object 
 5   block_percentage                        3844 non-null   float64
 6   blocks                                  3844 non-null   object 
 7   conference                              3844 non-null   object 
 8   conference_losses                       3838 non-null   object 
 9   conference_wins                         3838 non-null   object 
 10  defensive_rebounds                      3844 